In [2]:
import pandas as pd
from biblib import Entry
import pybtex as pbt
import math
import numpy as np
import re

## Load data needed to generate tables

In [33]:
path = '~/Downloads/Simple Decision Rules Give High Accuracy for Detecting Social Bots on Benchmark Datasets - Bot detection papers(14).tsv'
#path = 'metrics.tsv'

df = pd.read_csv(path, sep='\t')
df.fillna("", inplace=True)

In [55]:
dataset_df_path = '~/Downloads/Simple Decision Rules Give High Accuracy for Detecting Social Bots on Benchmark Datasets - datasets(8).tsv'

dataset_df = pd.read_csv(dataset_df_path, sep='\t')
dataset_df.fillna("", inplace=True)


In [5]:
scores_path = '~/work/repo/bot-detection/scores.csv'
sdt_df = pd.read_csv(scores_path)


## Generate bibliography methods.bib

In [56]:
bib = pbt.database.BibliographyData()

def add_bib_entries(df):
    for row in df.to_dict(orient="records"):
        #print(row)
        if row['bibtex_id'] in bib.entries.keys():
            continue
        if 'analyzed?' in row:
            if not row['analyzed?']:
                continue
        if not row['bibtex_id']:
            continue
        inputdict = {
            'author': row['authors'],
            'title': row['title'],
            'year': str(int(row['year']))
        }

        if row['conference?']:
            inputdict.update({
                'booktitle': row['booktitle'],
                'pages': row['pages'],
            })
            if row['booktitle']:
                inputdict['booktitle'] = row['booktitle'] 
            type_ = 'inproceedings'
        else:
            inputdict['journal'] = row['journal']
            if row['volume']:
                inputdict['volume'] = str(int(row['volume']))
            if row['number']:
                inputdict['number'] = str(int(row['number']))
            type_ = 'article'
        if row['publisher']:
            inputdict['publisher'] = row['publisher']
        if row['doi']:
            inputdict['doi'] = row['doi']
        if row['pages']:
            inputdict['pages'] = row['pages']
        entry = pbt.database.Entry(type_=type_, fields=inputdict)
        bib.add_entry(entry=entry, key=row['bibtex_id'])

add_bib_entries(df)
add_bib_entries(dataset_df)

bib.to_file("methods.bib")

## Generate table dataset -> paper that uses it

In [7]:
dataset_dict = {}

for row in df.to_dict(orient="records"):
    datasets = row['dataset(s) used'].split("; ")
    for d in datasets:
        if d in dataset_dict:
            dataset_dict[d].append(row['bibtex_id'])
        else:
            dataset_dict[d] = [row['bibtex_id']]

In [8]:
for k,v in dataset_dict.items():
    impl_papers = ", ".join(dataset_dict[k])

    cite_as = '\\cite{' + impl_papers + '} \\\\'
    dataset_name = k
    print(dataset_name+ " & " + cite_as)

twibot-2020 & \cite{feng2022heterogeneity-aware, , , feng2021botrgcn, geng2021satar, dehghan2018detecting} \\
cresci-rtbust-2019 & \cite{guo2022social, , yang2020scalable, sayyadiharikandeh2020detection, , , , mazza2019rtbust} \\
botometer-feedback-2019 & \cite{guo2022social, , yang2020scalable, sayyadiharikandeh2020detection} \\
gilani-2017 & \cite{guo2022social, dimitriadis2021social, , yang2020scalable, gilani2020classification, sayyadiharikandeh2020detection, , echeverria2018lobo} \\
cresci-stock-2018 & \cite{guo2022social, dimitriadis2021social, , , yang2020scalable, sayyadiharikandeh2020detection} \\
midterm-2018 & \cite{guo2022social, dimitriadis2021social, , yang2020scalable, sayyadiharikandeh2020detection, } \\
cresci-2015 & \cite{gonzalez2022the, , dimitriadis2021social, , stella2019influence, echeverria2018lobo, cresci2015fame} \\
cresci-2017 & \cite{gonzalez2022the, thavasimani2022a, heidari2021an, ilias2021detecting, geng2021satar, dimitriadis2021social, , yang2020scalable

## Generate table for dataset, #people/bots, description

In [54]:
for row in dataset_df.to_dict(orient="records"):
    if not row['analyzed?']:
        continue
    name = "\\data{" + row['dataset name'] + "}"
#     if row['aggregated benchmark dataset']:
#         name = name + "\\data{" + f" ({row['aggregated benchmark dataset']})" + "}"
        
    num_humans = int(row['# humans we have']) if row['# humans we have']!="" else '-'
    num_bots = int(row['# bots we have']) if row['# bots we have']!="" else '-'
    #print(name + " & \\cite{" + row['bibtex_id'] + "} & " + f"{num_users} & {prop_bots:.2f} \\\\")
    print(
        name + \
        " & \\cite{" + \
        row['bibtex_id'] + \
        "} " + \
        f"& {num_humans} & {num_bots} & {row['description']}\\\\"
    )
    if row['aggregated benchmark dataset']:
        print("\\data{" + f" ({row['aggregated benchmark dataset']})" + "} \\\\")

\data{botometer-feedback-2019} & \cite{yang2019arming} & 380 & 139 & Accounts misclassified by public, manually annotated by PhD student.\\
\data{botwiki-2019} & \cite{yang2020scalable} & 0 & 698 & Self-identified bot accounts.\\
\data{caverlee-2011} & \cite{lee2011a} & 19276 & 22223 & Bots collected by honeypot.\\
\data{ (pan-2019)} \\
\data{celebrity-2019} & \cite{yang2019arming} & 5918 & 0 & Celebrity accounts.\\
\data{the-fake-project-2015} & \cite{cresci2015fame} & 469 & 0 & Followers of @TheFakeProject who completed CAPTCHA.\\
\data{ (cresci-2015;  pan-2019)} \\
\data{elezioni-2015} & \cite{cresci2015fame} & 1488 & 0 & Twitter users who used hashtag #elezioni2013, excluding politicians, media and parties; manually verified by sociologists.\\
\data{ (cresci-2015; pan-2019)} \\
\data{fake-followers-2015} & \cite{cresci2017the} & 0 & 3351 & Fake followers.\\
\data{ (cresci-2017; cresci-2015;  pan-2019)} \\
\data{genuine-accounts-cresci-2017} & \cite{cresci2017the} & 3474 & 0 & Rando

## Generate table for dataset -> sdt/sota scores

In [34]:
def get_score(score, dataset_name):
    if score == "":
        return -1
    match = re.search(f"all: ([0-9]*(\.\d+)?)", score)
    if match:
        return -1
    match = re.search(f"{dataset_name}: ([0-9]*(\.\d+)?)", score)
    if match:
        return float(match.group(1))
    return float(score)

In [35]:
def get_max_score(df, dataset_name, metric):
    scores = df[df['dataset(s) used'].str.contains(dataset_name)][metric].map(lambda x: get_score(x, dataset_name))
    max_score_ind = scores.idxmax()
    return scores.loc[max_score_ind], df.at[max_score_ind, 'bibtex_id']


In [36]:
dataset_names = [
    'botometer-feedback-2019',
    'caverlee-2011',
    'cresci-2015',
    'cresci-2017',
    'cresci-rtbust-2019',
    #'cresci-stock-2018',
    #'yang-2013',
    'gilani-2017',
    'midterm-2018',
    'pan-2019',
    'twibot-2020',
    'varol-2017'
]

score_dict = {}

for name in dataset_names:
    score_dict[name] = {
        'accuracy': get_max_score(df, name, 'accuracy'),
        'f1': get_max_score(df, name, 'f1')
    }

In [37]:
score_dict

{'botometer-feedback-2019': {'accuracy': (0.8108, 'guo2022social'),
  'f1': (0.6977, 'guo2022social')},
 'caverlee-2011': {'accuracy': (0.9862, 'lee2011a'),
  'f1': (0.986, 'lee2011a')},
 'cresci-2015': {'accuracy': (0.991, 'cresci2015fame'),
  'f1': (0.991, 'cresci2015fame')},
 'cresci-2017': {'accuracy': (0.9981, 'kudugunta2018deep'),
  'f1': (1.0, 'kudugunta2018deep')},
 'cresci-rtbust-2019': {'accuracy': (0.9304, 'mazza2019rtbust'),
  'f1': (0.8687, 'mazza2019rtbust')},
 'gilani-2017': {'accuracy': (0.8644, 'gilani2020classification'),
  'f1': (0.836, 'gilani2020classification')},
 'midterm-2018': {'accuracy': (0.964, 'antenore2022a'),
  'f1': (0.9413, 'ng2023botbuster')},
 'pan-2019': {'accuracy': (0.9509, 'geng2021satar'),
  'f1': (0.951, 'geng2021satar')},
 'twibot-2020': {'accuracy': (0.8664, 'feng2022heterogeneity-aware'),
  'f1': (0.8821, 'feng2022heterogeneity-aware')},
 'varol-2017': {'accuracy': (-1, 'dimitriadis2021social'),
  'f1': (0.7306, 'kosmajac2019twitter')}}

In [41]:
max_depth = 5
tolerance = 0.025

for k, v in score_dict.items():
    #print(k)
    if k == 'varol-2017':
        continue
    accuracy_sota = float(v['accuracy'][0])
    f1_sota = float(v['f1'][0])
    
    row = sdt_df[sdt_df['name'] == k].to_dict(orient="records")[0]
    accuracies = [row[f'a{i}'] for i in range(1, max_depth+1)]
    a_max_ind = np.argmax(accuracies)
    f1s = [row[f'f{i}'] for i in range(1, max_depth+1)]
    f_max_ind = np.argmax(f1s)
    accuracy_sdt = accuracies[a_max_ind]
    f1_sdt = f1s[f_max_ind]
    
    for i, acc in enumerate(accuracies):
        if accuracy_sdt - acc <= tolerance:
            a_max_ind = i
            accuracy_sdt = acc
            break
    for i, f in enumerate(f1s):
        if f1_sdt - f <= tolerance:
            f_max_ind = i
            f1_sdt = acc
            break
    
    
    
    accuracy_diff = accuracy_sdt - accuracy_sota
    f1_diff = f1_sdt - f1_sota
        
    print(f"{k} & {accuracy_sdt:0.2f}" \
          + " \\textit{" \
          + f"({a_max_ind+1})" \
          + "} & " \
          + f"{f1_sdt:0.2f} " \
          " \\textit{" \
          + f"({f_max_ind+1})" \
          + "} & " \
          + f"{accuracy_sota:0.2f}" \
          + " \\cite{" + f"{v['accuracy'][1]}" \
          + "} & " \
          + f"{f1_sota:0.2f}" \
          + " \\cite{" \
          + f"{v['f1'][1]}" + "} & " \
          + f"{accuracy_diff:0.2f} & {f1_diff:0.2f} \\\\")

botometer-feedback-2019 & 0.78 \textit{(4)} & 0.78  \textit{(5)} & 0.81 \cite{guo2022social} & 0.70 \cite{guo2022social} & -0.03 & 0.08 \\
caverlee-2011 & 0.93 \textit{(4)} & 0.93  \textit{(4)} & 0.99 \cite{lee2011a} & 0.99 \cite{lee2011a} & -0.06 & -0.06 \\
cresci-2015 & 0.97 \textit{(3)} & 0.97  \textit{(3)} & 0.99 \cite{cresci2015fame} & 0.99 \cite{cresci2015fame} & -0.02 & -0.02 \\
cresci-2017 & 0.98 \textit{(1)} & 0.98  \textit{(1)} & 1.00 \cite{kudugunta2018deep} & 1.00 \cite{kudugunta2018deep} & -0.02 & -0.02 \\
cresci-rtbust-2019 & 0.72 \textit{(4)} & 0.72  \textit{(1)} & 0.93 \cite{mazza2019rtbust} & 0.87 \cite{mazza2019rtbust} & -0.21 & -0.15 \\
gilani-2017 & 0.81 \textit{(1)} & 0.81  \textit{(2)} & 0.86 \cite{gilani2020classification} & 0.84 \cite{gilani2020classification} & -0.05 & -0.03 \\
midterm-2018 & 0.97 \textit{(1)} & 0.97  \textit{(1)} & 0.96 \cite{antenore2022a} & 0.94 \cite{ng2023botbuster} & 0.01 & 0.03 \\
pan-2019 & 0.93 \textit{(3)} & 0.93  \textit{(3)} & 0.95 